In [1]:
%env ARTS_INCLUDE_PATH=/scratch/uni/u237/users/mprange/arts/controlfiles/
%env ARTS_BUILD_PATH=/scratch/uni/u237/users/mprange/arts/build/

%matplotlib inline
import numpy as np
from typhon.arts.workspace import Workspace, arts_agenda
from typhon.physics import wavenumber2frequency

env: ARTS_INCLUDE_PATH=/scratch/uni/u237/users/mprange/arts/controlfiles/
env: ARTS_BUILD_PATH=/scratch/uni/u237/users/mprange/arts/build/
Loading ARTS API from: /scratch/uni/u237/users/mprange/arts/build/src/libarts_api.so


# Load generic agendas

In [2]:
ws = Workspace(verbosity = 0)
ws.execute_controlfile("general/general.arts")
ws.execute_controlfile("general/continua.arts")
ws.execute_controlfile("general/agendas.arts")
ws.execute_controlfile("general/planet_earth.arts")
ws.Copy(ws.abs_xsec_agenda, ws.abs_xsec_agenda__noCIA)

# Load f_grid

In [3]:
h2o_band = np.arange(wavenumber2frequency(1185 * 100), wavenumber2frequency(1405 * 100), 7.5e9)
co2_band = np.arange(wavenumber2frequency(595 * 100), wavenumber2frequency(755 * 100), 7.5e9)
ws.f_grid = np.concatenate((co2_band, h2o_band))

# Load batch_atm_fields

In [6]:
ws.ReadXML(ws.batch_atm_fields_compact,
           "/scratch/uni/u237/users/mprange/arts/controlfiles/testdata/garand_profiles.xml.gz")
ws.batch_atm_fields_compactAddConstant(name="abs_species-O2",
                                       value=0.2095,
                                       condensibles=["abs_species-H2O"])
ws.batch_atm_fields_compactAddConstant(name="abs_species-N2",
                                       value=0.7808,
                                       condensibles=["abs_species-H2O"])

# Set absorption species and load line data

In [6]:
def read_iasi_abs_lines(ws, spectral_limit1, spectral_limit2, spectral_str):
    """
    Read absorption lines from HITRAN catalogue into workspace for given spectral range, which can be
    in units of frequency, wavelength or wavenumber.
    :param ws: ARTS workspace object
    :param spectral_limit1: lower spectral limit
    :param spectral_limit2: upper spectral limit
    :param spectral_str: Can be "frequency", "wavelength" or "wavenumber"
    :return:
    """
    if spectral_str == "wavelength":
        spectral_limit2 = wavelength2frequency(spectral_limit1 * 1e-6)
        spectral_limit1 = wavelength2frequency(spectral_limit2 * 1e-6)
    elif spectral_str == "wavenumber":
        spectral_limit1 = wavenumber2frequency(spectral_limit1 * 100)
        spectral_limit2 = wavenumber2frequency(spectral_limit2 * 100)

    # define absorbing species and load lines for given frequency range from HITRAN
    ws.abs_speciesSet(species=["H2O, H2O-SelfContCKDMT252, H2O-ForeignContCKDMT252",
                               "O2, O2-CIAfunCKDMT100",
                               "N2, N2-CIAfunCKDMT252, N2-CIArotCKDMT252",
                               "O3",
                               "CO2, CO2-CKDMT252"])
    ws.abs_lineshapeDefine(shape='Voigt_Kuntz6',
                           forefactor='VVH',
                           cutoff=750e9)
    # Read HITRAN catalog
    ws.abs_linesReadFromSplitArtscat(
        basename='/scratch/uni/u237/data/catalogue/hitran/hitran_split_artscat5/',
        fmin=spectral_limit1,
        fmax=spectral_limit2)
    return ws
    

In [5]:
ws = read_iasi_abs_lines(ws, 595, 1405, "wavenumber")
ws.abs_lines_per_speciesCreateFromLines()

# Do the absorption lookup table calculation

In [8]:
ws.atmosphere_dim = 1  # for 1DVAR
ws.abs_lookupSetupBatch()
ws.abs_xsec_agenda_checkedCalc()

ws.abs_lookupCalc()

In [11]:
ws.WriteXML("binary", ws.abs_lookup, "a_priori/abs_lookup_iasi.xml")

 # Try loading the lookup table

In [10]:
ws.ReadXML(ws.abs_lookup, "a_priori/abs_lookup_iasi.xml")
ws.abs_speciesSet(species=["H2O, H2O-SelfContCKDMT252, H2O-ForeignContCKDMT252",
                           "O2, O2-CIAfunCKDMT100",
                           "N2, N2-CIAfunCKDMT252, N2-CIArotCKDMT252",
                           "O3",
                           "CO2, CO2-CKDMT252"])
ws.abs_lineshapeDefine(shape='Voigt_Kuntz6',
                       forefactor='VVH',
                       cutoff=750e9)
ws.abs_lookupAdapt()

In [34]:
ws.ReadXML(ws.batch_atm_fields_compact,
           "/scratch/uni/u237/users/mprange/arts/controlfiles/testdata/garand_profiles.xml.gz")
ws.batch_atm_fields_compactAddConstant(name="abs_species-O2",
                                       value=0.2095,
                                       condensibles=["abs_species-H2O"])
ws.batch_atm_fields_compactAddConstant(name="abs_species-N2",
                                       value=0.7808,
                                       condensibles=["abs_species-H2O"])
ws.Extract(ws.atm_fields_compact,
           ws.batch_atm_fields_compact,
           0)
ws.atm_fields_compact.value.grids[1]

array([1.01325e+05, 1.00543e+05, 9.85880e+04, 9.57440e+04, 9.22460e+04,
       8.82800e+04, 8.39950e+04, 7.95090e+04, 7.49120e+04, 7.02730e+04,
       6.56430e+04, 6.10600e+04, 5.65540e+04, 5.21460e+04, 4.78540e+04,
       4.36950e+04, 3.96810e+04, 3.58280e+04, 3.21500e+04, 2.86600e+04,
       2.53710e+04, 2.22940e+04, 1.94360e+04, 1.67950e+04, 1.43840e+04,
       1.22040e+04, 1.02050e+04, 8.51800e+03, 6.99700e+03, 5.67300e+03,
       4.52900e+03, 3.55100e+03, 2.72600e+03, 2.04000e+03, 1.48100e+03,
       1.03700e+03, 6.95000e+02, 4.41000e+02, 2.61000e+02, 1.42000e+02,
       6.90000e+01, 2.90000e+01, 1.00000e+01])